In [1]:
from river_torch.anomaly import Autoencoder
from river import metrics
from river.datasets import CreditCard
from torch import nn
import math

In [3]:
dataset = CreditCard().take(5000)
metric = metrics.ROCAUC()

def get_fc_ae(n_features):
    latent_dim = math.ceil(n_features / 2)
    return nn.Sequential(
        nn.Linear(n_features, latent_dim),
        nn.LeakyReLU(),
        nn.Linear(latent_dim, n_features),
    )

model = Autoencoder(build_fn=get_fc_ae, lr=0.01)

for x,y in dataset:
    score = model.score_one(x)
    metric.update(y_true=y, y_pred=score)
    model.learn_one(x=x)
print(f'ROCAUC: {metric.get()}')

ROCAUC: -0.0
